In [ ]:
using TracyWidomBeta,LinearAlgebra,Statistics,SparseArrays,Distributions,Trapz,ApproxFun,SpecialFunctions
using RandomMatrices,Plots,LaTeXStrings,TypedTables,StatsBase,FFTW

## Finite-difference discretization

The following plots show the absolute stability region of BDF5 for $\beta=2$ along with $z\equiv \Delta x \lambda$, where $\lambda$ is the eigenvalue of $T(\beta,\pmb{\theta}_{M},h)+U(\beta,x,\pmb{\theta}_{M},h)$ from the finite-difference discretization. $\Delta x=-0.001$ and $M=1000$. 

In [ ]:
ρ = (α,z) -> (z.^(length(α)-1:-1:0))'*α
σ = (β,z) -> (z.^(length(β)-1:-1:0))'*β
R = (α,β,z) -> ρ(α,z)/σ(β,z)
function find_roots(c) # supposing that the leading order coefficient is 1
    # c contains the remaining coefficients
    r = length(c)
    A = zeros(Complex{Float64},r,r)
    A[1,:] = -c
    A[2:end,1:end-1] = A[2:end,1:end-1] + I # add identity matrix to lower-left block
    return eigvals(A)
end
function compute_roots(α,β,z)
    r = length(α)-1
    c = α-z*β
    if α[1]-z*β[1] ≈ 0.
        λ = find_roots(c[3:end]/c[2])
    else
        λ = find_roots(c[2:end]/c[1]) # let's suppose that first and second coefficients don't vanish simultaneously
    end
    return λ
end
function check_condition(λ)
    if maximum(abs.(λ)) > 1
        return 0
    else
        for i = 1:length(λ)
            if abs(λ[i]) ≈ 1. && sum(map(t -> λ[i] ≈ t,λ)) > 1
                return 0
            end
        end
    end
    return 1
end
function root_condition(α,β,z)
    return compute_roots(α,β,z) |> check_condition
end

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-10.,20.];
    yrange=[-15.,15.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end

In [ ]:
# Generate the matrices
β=2;
M_f=1000;
mgrid=(n,L) -> L*(1:n)/n;
θ=mgrid(M_f,pi);
M_s=10; #just some random value
h=(1/M_f)*pi;
method="finite";
l=10;
(A,B)=TracyWidomBeta.matrix_gen(β;method,M_f,M_s,h,θ,l);

# Plot the absolute stability region
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-10.,20.];
    yrange=[-15.,15.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [137/60,-5,5,-200/60,75/60,-1/5];
β = [1,0,0,0,0,0];
bdf5_fds=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf5_fds=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf5_fds

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-1.,1.];
    yrange=[-1.,1.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [137/60,-5,5,-200/60,75/60,-1/5];
β = [1,0,0,0,0,0];
bdf5_fdl=convergence_stability(α,β);
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf5_fdl=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf5_fdl

The following plots show the absolute stability region of BDF6 for $\beta=2$ along with $z\equiv \Delta x \lambda$, where $\lambda$ is the eigenvalue of $T(\beta,\pmb{\theta}_{M},h)+U(\beta,x,\pmb{\theta}_{M},h)$ from the finite-difference discretization. $\Delta x=-0.001$ and $M=1000$. 

In [ ]:
# Generate the matrices
β=2;
M_f=1000;
mgrid=(n,L) -> L*(1:n)/n;
θ=mgrid(M_f,pi);
M_s=10; #just some random value
h=(1/M_f)*pi;
method="finite";
l=20;
(A,B)=TracyWidomBeta.matrix_gen(β;method,M_f,M_s,h,θ,l);

# Plot the absolute stability region
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-10.,30.];
    yrange=[-25.,25.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [147/60,-6,45/6,-400/60,225/60,-72/60,1/6];
β = [1,0,0,0,0,0,0];
bdf6_fds=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf6_fds=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf6_fds

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-1.,1.];
    yrange=[-1.,1.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [147/60,-6,45/6,-400/60,225/60,-72/60,1/6];
β = [1,0,0,0,0,0,0];
bdf6_fdl=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf6_fdl=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf6_fdl

## Spectral discretization

The following plots show the absolute stability region of BDF5 for $\beta=2$ along with $z\equiv \Delta x \lambda$, where $\lambda$ is the eigenvalue of $A+xB$ from the spectral discretization. $\Delta x=-0.001$ and $M=8000$. 

In [ ]:
# Generate the matrices
β=2;
M_f=1000;#just some random value
mgrid=(n,L) -> L*(1:n)/n;
θ=mgrid(M_f,pi);
M_s=8000;
h=(1/M_f)*pi;
method="spectral";
l=10;
(A,B)=TracyWidomBeta.matrix_gen(β;method,M_f,M_s,h,θ,l);

# Plot the absolute stability region
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-10.,20.];
    yrange=[-15.,15.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [137/60,-5,5,-200/60,75/60,-1/5];
β = [1,0,0,0,0,0];
bdf5_sds=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 200, replace = false)
    bdf5_sds=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf5_sds

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-1.,1.];
    yrange=[-1.,1.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [137/60,-5,5,-200/60,75/60,-1/5];
β = [1,0,0,0,0,0];
bdf5_sdl=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf5_sdl=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf5_sdl

The following plots show the absolute stability region of BDF6 for $\beta=2$ along with $z\equiv \Delta x \lambda$, where $\lambda$ is the eigenvalue of $A+xB$ from the spectral discretization. $\Delta x=-0.001$ and $M=8000$. 

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-10.,30.];
    yrange=[-25.,25.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [147/60,-6,45/6,-400/60,225/60,-72/60,1/6];
β = [1,0,0,0,0,0,0];
bdf6_sds=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf6_sds=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf6_sds

In [ ]:
function convergence_stability(α,β)
    θ = 0:0.01(1+rand()/10):2*π # random perturbation to avoid singularities
    z = map(t -> R(α,β,exp(1im*t)),θ);
    xrange=[-1.,1.];
    yrange=[-1.,1.];
    contourf(xrange[1]:0.01:xrange[2],yrange[1]:0.01(1+rand()/10):yrange[2],(x,y)-> root_condition(α,β,x+1im*y),colorbar=false,
        xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
    plot!(real(z),imag(z),xlim=xrange,ylim=yrange,aspectratio=1,legend=false,lw=5,linecolor=:orange,xtickfontsize=15,ytickfontsize=15,
        legendfontsize=15)
end
α = [147/60,-6,45/6,-400/60,225/60,-72/60,1/6];
β = [1,0,0,0,0,0,0];
bdf6_sdl=convergence_stability(α,β);

# Plot z
x0 = 13.0;
xN = -10.0;
Δx = -0.001;
for x=x0:-1:xN
    C=A+x*B;
    Ei=Δx*eigvals(Matrix(C));
    E=StatsBase.sample(Ei, 500, replace = false)
    bdf6_sdl=Plots.scatter!(E,c=:white,lw=0.005,xlabel="",ylabel="",xtickfontsize=15,ytickfontsize=15,legendfontsize=15)
end
bdf6_sdl